In [18]:
from pathlib import Path

def find_repo_root(start: Path | None = None) -> Path:
    """
    Walk upwards from `start` (or CWD) until we find a repo marker.
    Works in notebooks where __file__ is not available.
    """
    start = (start or Path.cwd()).resolve()
    markers = ("pyproject.toml", ".git", "setup.cfg", "requirements.txt")
    for p in (start, *start.parents):
        if any((p / m).exists() for m in markers):
            return p
    # last resort: assume current folder is repo root
    return start

REPO_ROOT = find_repo_root()
OUT_DIR = REPO_ROOT / "data" / "signals"
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("CWD:", Path.cwd())
print("REPO_ROOT:", REPO_ROOT)
print("OUT_DIR:", OUT_DIR)

CWD: r:\scanner\books
REPO_ROOT: C:\Users\MaartenEnde\Repos\scanner
OUT_DIR: C:\Users\MaartenEnde\Repos\scanner\data\signals


In [22]:
from scanner_bot.core.ports import BarStore
from scanner_bot.infra.db.sqllite_barstore import SQLiteBarStore

DB_PATH = Path("data/db/scanner.db")

store = SQLiteBarStore(DB_PATH)

print(store.list_bars('AAPL', '1d', None, None, 100))




[]
